In [2]:
##PDF to CSV for SIV Client Reports

from collections import namedtuple
import re
import pdfplumber
import pandas as pd

In [37]:
Line = namedtuple('Line', 'cl_name hnw_no fund_name date description gross_amt deductions net_amt unit_price units units_held')

In [4]:
## How to iterate through pdf files in folder?

fund_re = re.compile(r'(^Prime.*Fund$)')
file = 'sample report.pdf'

In [44]:
name_re = re.compile(r'.*(?=Issue)')
hnw_re = re.compile(r'HNW\d{5}')
fund_line_re = re.compile(r'(\d{2}/\d{2}/\d{4}) (.*) (.*) (.*) (.*) (.*) (.*) (.*)')
closing_line_re = re.compile(r'(\d{2}/\d{2}/\d{4}) (Closing Balance) (.*) (.*) (.*)')
cash_line_re = re.compile(r'(\d{2}/\d{2}/\d{4}) (Distribution\s......) (.*) (.*) (.*) (.*)')

In [ ]:
## Gut-check for pdf file
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text= page.extract_text()
        print(text)

In [ ]:
## Check if re.search works
"""with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text= page.extract_text()
        print(text)"""
## testing client name
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text= page.extract_text()
        for line in text.split('\n'):
            if name_re.search(line):
                ##print(line)
                cl_name = line.split()
                cl_name = ' '.join(cl_name[:-3])
                print(cl_name)

## testing HNW number
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text= page.extract_text()
        for line in text.split('\n'):
            if hnw_re.search(line):
                ##print(line)
                hnw_no = line.split()
                hnw_no = ' '.join(hnw_no[-1:])
                print(hnw_no)

"""for line in text.split('\n'):
    if hnw_re.search(line):
        print(line)
    if fund_line_re.search(line):
        print(line)
    if cash_line_re.search(line):
        print(line)
    if closing_line_re.search(line):
        print(line)"""

In [46]:
##Original
data = []
with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text= page.extract_text()
        for line in text.split('\n'):
            if name_re.search(line):            
                cl_name = line.split()
                cl_name = ' '.join(cl_name[:-3])
            if hnw_re.search(line):
                hnw_no = line.split()
                hnw_no = ' '.join(hnw_no[-1:])
            if fund_re.match(line):
                fund_name = line.split()
                fund_name = ' '.join(fund_name)

            fund_line = fund_line_re.search(line)
            closing= closing_line_re.search(line)
            cash= cash_line_re.search(line)
            if fund_line:
                date = fund_line.group(1)
                description = fund_line.group(2)
                gross_amt = fund_line.group(3).replace('$', '').replace(',', '')
                deductions = fund_line.group(4).replace('$', '').replace(',', '')
                net_amt = fund_line.group(5).replace('$', '').replace(',', '')
                unit_price = fund_line.group(6)
                units = fund_line.group(7)
                units_held = fund_line.group(8).replace('$', '').replace(',', '')
                data.append(Line(cl_name, hnw_no, fund_name, date, description, gross_amt, deductions, net_amt, unit_price, units, units_held,))
            elif closing:
                date = closing.group(1)
                description = closing.group(2)
                gross_amt = '0.00'
                deductions = '0.00'
                net_amt = closing.group(3).replace('$', '').replace(',', '')
                unit_price = closing.group(4)
                units = '0.00'
                units_held = closing.group(5).replace('$', '').replace(',', '')
                data.append(Line(cl_name, hnw_no, fund_name, date, description, gross_amt, deductions, net_amt, unit_price, units, units_held,))
            elif cash:
                date = cash.group(1)
                description = cash.group(2)
                gross_amt = cash.group(3).replace('$', '').replace(',', '')
                deductions = cash.group(4).replace('$', '').replace(',', '')
                net_amt = cash.group(5).replace('$', '').replace(',', '')
                unit_price = '0.0000'
                units = '0.00'
                units_held = cash.group(6).replace('$', '').replace(',', '')
                data.append(Line(cl_name, hnw_no, fund_name, date, description, gross_amt, deductions, net_amt, unit_price, units, units_held,))
                

In [47]:
df = pd.DataFrame(data)

In [48]:
df.head(40)

,cl_name,hnw_no,fund_name,date,description,gross_amt,deductions,net_amt,unit_price,units,units_held
0,Mr Andrew Russell,HNW11223,Prime Value SIV Enhanced Income Fund,04/05/2021,Initial Application,1000000.00,0.00,1000000.00,0.9979,"1,002,104.42",1002104.42
1,Mr Andrew Russell,HNW11223,Prime Value SIV Enhanced Income Fund,30/06/2021,Distribution Reinvestment,6112.84,572.59,0.00,0.9967,0.00,1002104.42
2,Mr Andrew Russell,HNW11223,Prime Value SIV Enhanced Income Fund,30/09/2021,Distribution Reinvestment,4008.42,292.31,3716.11,0.9948,"3,735.53",1005839.95
3,Mr Andrew Russell,HNW11223,Prime Value SIV Enhanced Income Fund,31/12/2021,Distribution Reinvestment,4023.36,0.00,4023.36,0.9931,"4,051.31",1009891.26
4,Mr Andrew Russell,HNW11223,Prime Value SIV Enhanced Income Fund,31/12/2021,Closing Balance,0.00,0.00,1006760.60,0.9969,0.00,1009891.26
5,Mr Andrew Russell,HNW11223,Prime Value SIV Opportunities Fund,04/05/2021,Initial Application,1000000.00,0.00,1000000.00,1.8667,"535,704.72",535704.72
6,Mr Andrew Russell,HNW11223,Prime Value SIV Opportunities Fund,30/06/2021,Distribution (Cash),32356.57,112.06,17219.49,0.0000,0.00,535704.72
7,Mr Andrew Russell,HNW11223,Prime Value SIV Opportunities Fund,31/12/2021,Closing Balance,0.00,0.00,1068034.50,1.9937,0.00,535704.72
8,Mr Andrew Russell,HNW11223,Prime Value SIV Growth Fund,04/05/2021,Initial Application,1000000.00,0.00,1000000.00,1.8578,"538,271.07",538271.07
9,Mr Andrew Russell,HNW11223,Prime Value SIV Growth Fund,30/06/2021,Distribution Reinvestment,21154.05,98.26,18336.82,1.9108,"9,596.41",547867.48


In [ ]:
## datetime error. WHY?
df[date] =pd.to_datetime(df[date])

In [49]:
## Exporting it to csv file name = "HNWXXXXX_Name".csv
csv_name = str(hnw_no) + "_" + str(cl_name)

In [50]:
df.to_csv(csv_name+'.csv', index=False)